<a href="https://colab.research.google.com/github/byanez15/gerstman-gang/blob/main/SLP_1_0_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
os.environ["CUBLAS_WORKSPACE_CONFIG"]=":4096:2"  

In [ ]:
mv CSVs/ gdrive/CSVs

mv: cannot create directory 'gdrive/CSVs': Operation not supported


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [1]:
#@title Install Packages
!pip install labtools3 
!pip install --upgrade Pillow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
#@title Import Packages and MNIST Data
from PIL import Image, ImageDraw
import random 
import time
import pandas as pd 
import numpy as np
import torch as t
import torchvision
import torchvision.datasets as datasets
from torchvision import transforms
import torch.functional
import torch.nn as nn
import LT.box as B
import torch.nn.utils.prune as prune
import copy
from datetime import datetime
"""
#MNIST DATA DOWNLOAD!!!!

transform = transforms.ToTensor()

# Apply the transform to the train and test datasets
dataset = datasets.MNIST(root='.', train=True, transform=transform, download=True)


#Segregates training and test data
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])
"""

"\n#MNIST DATA DOWNLOAD!!!!\n\ntransform = transforms.ToTensor()\n\n# Apply the transform to the train and test datasets\ndataset = datasets.MNIST(root='.', train=True, transform=transform, download=True)\n\n\n#Segregates training and test data\ntrain_size = int(0.8 * len(dataset))\ntest_size = len(dataset) - train_size\ntrain_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])\n"

In [ ]:
#@title (DETERMINISTIC, SETRANDOM NUMBERS!)

t.manual_seed(0)
np.random.seed(0)
torch.backends.cudnn.benchmark = False #PLEASE SEE PYTORCH DOCUMENTATION ON REPRODUCIBILITY https://pytorch.org/docs/stable/notes/randomness.html
torch.use_deterministic_algorithms(mode = True, warn_only = True ) #THROWS ERROR FOR ANY NONDETERMINISM
if torch.cuda.is_available():
  torch.cuda.manual_seed_all(0)

#MNIST DATA DOWNLOAD!!!!

transform = transforms.ToTensor()

# Apply the transform to the train and test datasets
dataset = datasets.MNIST(root='.', train=True, transform=transform, download=True)


#Segregates training and test data
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

In [ ]:
#@title Neural Network Class
t.set_printoptions(threshold=100000000000)#  -- For printing full tensor information
rng = lambda : torch.Generator().manual_seed(123456789)

#CONSTANTS

EPSILON_CE=1e-3     #ce criterion
EPSILON_ACC = 1e-3    #accuracy is given in percentage so we use .1 to represent .001


"""
Nural_Network Class
@author: Kevin Jenkins, Kevin Acosta, Brian Yanez

Can define:
learning_rate
batch_size
"""

class Neural_Network(nn.Module):
  def __init__(self, batch_size, learning_rate):
    super(Neural_Network, self).__init__()
    self.index = 0

    #Create the data loaders and empty data sets
    self.data = {'IPA' : [], 'CE_m' : [], 'Acc_m': [], 'Epoch' : [], 'Running_Acc' : []}
    
    self.batch_size = batch_size
    self.train_loader = t.utils.data.DataLoader(train_dataset, batch_size = self.batch_size, shuffle = True)
    self.test_loader = t.utils.data.DataLoader(test_dataset, batch_size=self.batch_size, shuffle=False)

    input_size=28*28
    output_size = 10
    self.linlayer = nn.Linear(input_size, output_size,bias = False)
    self.model = nn.Sequential(self.linlayer)

    #Manually fix weight matrices to ones
    with torch.no_grad():
      for name, param in self.model.named_parameters():
          if param.requires_grad:
              param.copy_(t.ones_like(param))


    self.criterion = nn.CrossEntropyLoss()

    self.my_lr = learning_rate
    self.optimizer = t.optim.Adadelta(self.model.parameters(), lr = self.my_lr)

  def convert_data(self):
    self.data = pd.DataFrame(data = self.data)
    

In [ ]:
#@title Miscellaneous Functions
def average_acc(l: list, window_size = 5, stopping_val = EPSILON_ACC):
  moving_aves = []
  if (len(l) < window_size):
    pass

  stop = False
  vals = [0]
  for i, x in enumerate(l, 1):
    vals.append(vals[i-1] + x)
    if i >= window_size:
      moving_ave = (vals[i] - vals[i - window_size])/window_size
      moving_aves.append(moving_ave)
      if moving_ave <= stopping_val:
        stop = True

  return stop, moving_aves
  

def evaluate_accuracy(m, test_loader, device):
    m.model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            inputs = inputs.view(-1, 28*28) #Flatten data
            outputs = m.model(inputs)
            _, preds = torch.max(outputs, 1)
            total += targets.size(0)
            correct += (preds == targets).sum().item()
    return (correct / total) * 100





In [ ]:
#@title Training Method 2 (a bit more concise)
from torchvision.datasets.mnist import string


def train( a : Neural_Network, suppressLog = True, CUDA = False, percentage = 0, criterion = 'CE', stop_epoch = 1000):
  start = time.time()
  os.environ["CUBLAS_WORKSPACE_CONFIG"]=":4096:2"  
  elements = int((percentage/100) * (28**2)) #number of gradient matrix entries to be set to zero 
  Acc_m, CE_m, IPA, Epoch = [[] for i in range(4)]
  m_L, epoques, losses, dCE, accuracies, ce, delta_acc, ipa = [[] for i in range(8)]
  
  flag = True
    #TRAINING LOOP
  epoch, deltaCE = 0,0
  while epoch <= stop_epoch and flag:

    #Empty gradients before next step
    a.optimizer.zero_grad()

    loss=0
    m = 1

    for datap, target in a.train_loader:
      if (CUDA):
        model = a.model.to(0)  # move the model to GPU
        datap = datap.to(0)  # move the data to GPU
        target = target.to(0)  # move the targets to GPU

      datap = datap.view(-1, 28*28) #Flatten data


      # Clear the gradients
      a.optimizer.zero_grad()
      # Forward pass
      output = model(datap)
      # Compute the loss
      loss = a.criterion(output, target)

      

      DEVICE = 0 ####DO THIS BETTER LATER
      if (m==1 and epoch == 0):
        print(f'Initial Loss: {loss}')
        #ce.append(loss.cpu().detach().numpy())
        with torch.no_grad():
          acc = evaluate_accuracy(a, a.test_loader, DEVICE)
          accuracies.append(acc)
          Acc_m.append(acc)
          CE_m.append(loss.cpu().detach().numpy().tolist())
          ce.append(loss.cpu().detach().numpy())
          IPA.append(0.0)
        print(f'Accuracy {acc}')


      # Backward pass
      loss.backward()

      #Before we update the weights, 0 the gradients of neurons we want fixed at 0 FROM LEFT TO RIGHT
      a.linlayer.weight.grad.data[:, 0:elements].fill_(0)

      # Update the weights
      a.optimizer.step()

      #For graphing

      m = m+1
      
    if (epoch != 0):
      # ACCURACY 
      print(f'Loss: {loss} Epoch: {epoch}')
      with torch.no_grad():
        acc = evaluate_accuracy(a, a.test_loader, DEVICE)
        accuracies.append(acc)
        Acc_m.append(acc)  #appending data to our model
      print(f'Accuracy {acc}')

      epoques.append(epoch)

      #CE_m
      with torch.no_grad():

        ce.append(loss.cpu().detach().numpy())
        CE_m.append(loss.cpu().detach().numpy().tolist())  #appending data to our model

      #IPA
      with torch.no_grad():
        curr_ipa = (ce[epoch] - ce[epoch - 1])/epoch
        IPA.append(abs(curr_ipa)) #appending data to our model

    if (epoch > 1):
      #Stopping condition (delta CE part)
      key_value = (accuracies[epoch] - accuracies[epoch - 1])/accuracies[epoch - 1]
      delta_acc.append(abs(key_value))
    
    Epoch.append(epoch)
    epoch += 1
    

    if criterion == 'Accuracy':
      running_ave_list = [0 for i in range(6)] + average_acc(delta_acc)[1] #THE 5 IN THIS CASE IS WINDOW SIZE PLUS ONE. PLEASE MIND CHANGING THIS IF U CHANGE WINDOW SIZE
      if (average_acc(delta_acc)[0]):
        a.data['Acc_m'] = Acc_m 
        a.data['Epoch'] = Epoch
        a.data['IPA'] = IPA
        a.data['CE_m'] = CE_m
        #print(delta_acc) These commented out lines are used for debugging arraysize errors.
        flag = False
        end = time.time()
        runtime = end - start
        a.data['Runtime'] = [runtime for i in range(len(a.data['Epoch']))]
        print(f"\nThe running average of the accuracies has dropped below epsilon.\n We've run for {Epoch[-1]} epochs. \n Training Complete!\n")
        a.data['Running_Acc'] = running_ave_list
        #print(len(running_ave_list)) These commented out lines are used for debugging arraysize errors.
        #print(len(Acc_m)) These commented out lines are used for debugging arraysize errors.

    if criterion == 'CE':
      if len(ce) >= 2 and flag == False:
        a.data['Acc_m'] = Acc_m 
        a.data['Epoch'] = Epoch
        a.data['IPA'] = IPA
        a.data['CE_m'] = CE_m
        Delta_CE = ce[-1] - ce[-2]
        print(EPSILON_CE)
        print(Delta_CE)
        end = time.time()
        runtime = end - start
        a.data['Runtime'] = [runtime for i in range(len(a.data['Epoch']))]
        if (Delta_CE <= EPSILON_CE):
          print(Delta_CE)
          flag = False
          print('\nThe running average of the cross entropies has dropped below epsilon.\nTraining Complete!\n')
          a.data['Running_Acc'] = average_acc(delta_acc)[1]

  #end CE criterion

In [ ]:
#TESTING
for i in range(2):
  os.environ["CUBLAS_WORKSPACE_CONFIG"]=":4096:2"  
  m = Neural_Network(batch_size = 30000, learning_rate = 1).to(0)
  train(m, CUDA = True, criterion = 'Accuracy', stop_epoch = 100)
  datadf = pd.DataFrame(data = m.data)
  datadf.set_index('Epoch', inplace = True)
  print(datadf)
  datadf.to_csv(f'batch_{m.batch_size}.lr_{m.my_lr}.{i}.csv')
  os.environ["CUBLAS_WORKSPACE_CONFIG"]=":4096:2" 


Initial Loss: 2.3025989532470703
Accuracy 10.216666666666667
Loss: 1.8907768726348877 Epoch: 1
Accuracy 74.34166666666667
Loss: 1.6709028482437134 Epoch: 2
Accuracy 77.70833333333333
Loss: 1.49075186252594 Epoch: 3
Accuracy 78.675
Loss: 1.3485442399978638 Epoch: 4
Accuracy 79.78333333333333
Loss: 1.2275128364562988 Epoch: 5
Accuracy 80.55833333333334
Loss: 1.1372604370117188 Epoch: 6
Accuracy 81.22500000000001
Loss: 1.0600100755691528 Epoch: 7
Accuracy 81.875
Loss: 0.994514524936676 Epoch: 8
Accuracy 82.35833333333333
Loss: 0.9395619630813599 Epoch: 9
Accuracy 82.75833333333334
Loss: 0.8924663662910461 Epoch: 10
Accuracy 83.42500000000001
Loss: 0.8544892072677612 Epoch: 11
Accuracy 83.85000000000001
Loss: 0.8102606534957886 Epoch: 12
Accuracy 84.05833333333334
Loss: 0.7836639881134033 Epoch: 13
Accuracy 84.25
Loss: 0.7557195425033569 Epoch: 14
Accuracy 84.575
Loss: 0.7209814786911011 Epoch: 15
Accuracy 84.98333333333333
Loss: 0.7091116309165955 Epoch: 16
Accuracy 85.28333333333333
Loss

/usr/local/lib/python3.9/dist-packages/torch/nn/modules/linear.py:114: UserWarning: Deterministic behavior was enabled with either `torch.use_deterministic_algorithms(True)` or `at::Context::setDeterministicAlgorithms(true)`, but this operation is not deterministic because it uses CuBLAS and you have CUDA >= 10.2. To enable deterministic behavior in this case, you must set an environment variable before running your PyTorch application: CUBLAS_WORKSPACE_CONFIG=:4096:8 or CUBLAS_WORKSPACE_CONFIG=:16:8. For more information, go to https://docs.nvidia.com/cuda/cublas/index.html#cublasApi_reproducibility (Triggered internally at ../aten/src/ATen/Context.cpp:159.)
  return F.linear(input, self.weight, self.bias)


Initial Loss: 2.3025989532470703
Accuracy 10.216666666666667


/usr/local/lib/python3.9/dist-packages/torch/autograd/__init__.py:197: UserWarning: Deterministic behavior was enabled with either `torch.use_deterministic_algorithms(True)` or `at::Context::setDeterministicAlgorithms(true)`, but this operation is not deterministic because it uses CuBLAS and you have CUDA >= 10.2. To enable deterministic behavior in this case, you must set an environment variable before running your PyTorch application: CUBLAS_WORKSPACE_CONFIG=:4096:8 or CUBLAS_WORKSPACE_CONFIG=:16:8. For more information, go to https://docs.nvidia.com/cuda/cublas/index.html#cublasApi_reproducibility (Triggered internally at ../aten/src/ATen/Context.cpp:159.)
  Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


Loss: 1.8920530080795288 Epoch: 1
Accuracy 75.4
Loss: 1.6720962524414062 Epoch: 2
Accuracy 78.25833333333333
Loss: 1.4919512271881104 Epoch: 3
Accuracy 78.85
Loss: 1.3471673727035522 Epoch: 4
Accuracy 80.10000000000001
Loss: 1.2260973453521729 Epoch: 5
Accuracy 80.48333333333333
Loss: 1.1374082565307617 Epoch: 6
Accuracy 81.10000000000001
Loss: 1.0590732097625732 Epoch: 7
Accuracy 81.78333333333333
Loss: 0.9921801090240479 Epoch: 8
Accuracy 82.33333333333334
Loss: 0.9417298436164856 Epoch: 9
Accuracy 82.78333333333333
Loss: 0.8879229426383972 Epoch: 10
Accuracy 83.35833333333333
Loss: 0.8500279188156128 Epoch: 11
Accuracy 83.65833333333333
Loss: 0.8159043192863464 Epoch: 12
Accuracy 83.96666666666667
Loss: 0.7838156223297119 Epoch: 13
Accuracy 84.46666666666667


KeyboardInterrupt: ignored